# Chapter 4. Relationships between Observations

The previous chapter discussed ways to measure relationships between variables, or the _columns_ of a `DataFrame`. This chapter is about how to measure relationships between observations, or the _rows_ of a `DataFrame`.

# Chapter 4.1 Distance Metrics

How do we measure how "similar" two observations are? To keep things simple, we will start with a data set where all the variables are quantitative: a data set about the chemical properties of 1599 red wines.

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.display.max_rows = 5

wines = pd.read_csv("/data301/data/winequality-red.csv", sep=";")
wines

To simplify things even further, let's start by just looking at just two variables: pH and free sulfur dioxide. We can make a scatterplot of these two variables.

In [ ]:
colors = pd.Series("lightgray", index=wines.index)
colors.loc[[1597, 1598]] = "red"

wines.plot.scatter(x="pH", y="free sulfur dioxide", c=colors)

How similar are the two observations represented by red points? The obvious way to measure similarity is to calculate the distance between the points. The smaller the distance, the more similar the observations.

<img src="distance.png">

Calculating the distance between two points is not as easy as it might seem. First, there is the question of how to define distance. In general, a **distance metric** can be any function of two observations, $ d({\bf x}, {\bf x'}) $, which is...

- non-negative: $d({\bf x}, {\bf x'}) \geq 0$ for all points ${\bf x}$ and ${\bf x'}$
- zero for any observation and itself: $d({\bf x}, {\bf x}) = 0$

Some people, particularly mathematicians, require that distance metrics satisfy additional properties, such as symmetry ($d({\bf x}, {\bf x'}) = d({\bf x'}, {\bf x})$) and the triangle inequality, but many distance "metrics" used in practice do not satisfy those properties. The two properties above are the only ones that are satisfied by virtually all distance metrics used in practice.

You are probably most familiar with **Euclidean distance**, which is the straight-line distance ("as the crow flies") between two points. The length of the straight-line can be calculated using the Pythagorean Theorem:

$$ d({\bf x}, {\bf x'}) = \sqrt{(x_1 - x'_1)^2 + (x_2 - x'_2)^2} $$

This formula generalizes to more than two variables. If we have $D$ variables, the formula becomes:

$$ d({\bf x}, {\bf x'}) = \sqrt{\sum_{j=1}^D (x_j - x'_j)^2} $$

Let's calculate the distance between the two observations above, using all of the variables in the data set.

In [ ]:
x = wines.loc[1597]
x_prime = wines.loc[1598]

x - x_prime

In [ ]:
(x - x_prime) ** 2

In [ ]:
np.sqrt(((x - x_prime) ** 2).sum())

To convince you that this is not the only way to calculate the distance between these two points, we can also consider [**taxicab distance**](https://en.wikipedia.org/wiki/Taxicab_geometry):

$$ d({\bf x}, {\bf x'}) = \sum_{j=1}^D |x_j - x'_j|. $$

Taxicab distance is so named because it measures how far a taxicab would have to drive to get from point A to point B. (Because of the layout of the streets, the taxicab cannot travel in a straight line from A to B.)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/08/Manhattan_distance.svg/283px-Manhattan_distance.svg.png">

In [ ]:
((x - x_prime).abs()).sum()

This is a noticeably different number!

You might want to know which distance metric is better. There is no easy answer to this question; it depends on the data and the application. Because differences are _squared_ in Euclidean distance, a large difference in _any_ variable will have a large effect on the Euclidean distance. On the other hand, taxicab distance treats all variables the same.

For example, consider the pair of points $(0, 0)$ and $(2, 2)$. Euclidean distance says that they are $\sqrt{2^2 + 2^2} = 2.8$ units apart, while Manhattan distance says that they are $2 + 2 = 4$ units apart. Don't compare these two numbers to each other. Instead, compare them to the distances for a different pair of points.

Now consider the pair of points $(0, 0)$ and $(3, 1)$. Would you consider these points to be closer or further than the pair above? Euclidean distance says that they are $\sqrt{3^2 + 1^2} \approx 3.2$ units apart (so further than before), while Manhattan distance still considers them $3 + 1 = 4$ units apart. So Euclidean distance penalizes observations more heavily for being off on any one variable. It prefers that differences be evenly distributed among the variables.

In general, distances can be defined between all kinds of observations, not just vectors of numbers. Later in this book, we will see how to calculate distances between strings, documents, and so on.

## The Importance of Standardization

Here's a quiz. There are two red points in the plot below, each labeled by its index in the `DataFrame`. Which one is closer to the blue point (using Euclidean distance, just based on these two variables alone)?

<img src="closer.png">

Let's calculate the distances.

In [ ]:
x_vars = ["pH", "free sulfur dioxide"]
x553 = wines.loc[553, x_vars]

In [ ]:
# Distance between 151 and 553
x151 = wines.loc[151, x_vars]
np.sqrt(((x151 - x553) ** 2).sum())

In [ ]:
# Distance between 94 and 553
x94 = wines.loc[94, x_vars]
np.sqrt(((x94 - x553) ** 2).sum())

Surprised by the answer? The scatterplot is deceiving because it automatically scales the variables to make all of the points fit into the limited space. In reality, the spread of the sulfur dioxide values is so much larger (on the order of tens) than the spread of the pH values (on the order of tenths).

The plot below shows the same data, drawn to scale. You can see that differences in sulfur dioxide dominate the distance metric; differences in pH hardly matter at all.

In [ ]:
colors = pd.Series("lightgray", index=wines.index)
colors.loc[[94, 151]] = "red"
colors.loc[553] = "blue"

wines.plot.scatter(x="pH", y="free sulfur dioxide", c=colors, 
                   xlim=(0, 70), ylim=(0, 70))

To obtain distances that agree more with our intuition---and that do not give undue weight to one variable---we transform the variables to be on the same scale. There are a few ways to do this:

- **standardizing**: subtract each variable by its mean, then divide by its standard deviation, 
$$ x_i \leftarrow \frac{x_i - \bar x}{s} $$
- **normalizing**: scale each variable to have length (or "norm") 1, 
$$ x_i \leftarrow \frac{x_i}{\sqrt{\sum_{i=1}^n x_i^2}} $$
- **min/max scaling**: scale each variable to take on values between [0, 1], 
$$x_i \leftarrow \frac{x_i - \min(x_1, ..., x_n)}{\max(x_1, ..., x_n) - \min(x_1, ..., x_n)}$$.

Let's first try standardizing.

In [ ]:
wines_s = (wines - wines.mean()) / wines.std()
wines_s

Notice that the resulting `DataFrame` is filled with negative values and values with magnitude less than 1. This makes sense if the variables now have mean 0 and SD 1.

The above command is deceptively simple. Notice that we actually subtracted a `DataFrame` by a `Series`, then divided the resulting `DataFrame` by another `Series`. We relied on the fact that `pandas` chooses the right axis along which to align the `Series`. To be more explicit, we could write:

In [ ]:
wines_s = wines.sub(wines.mean(), axis=1).divide(wines.std(), axis=1)
wines_s

Now let's recalculate distances using the standardized data.

In [ ]:
x_vars = ["pH", "free sulfur dioxide"]
x553 = wines_s.loc[553, x_vars]

In [ ]:
# Distance between 151 and 553, standardized data
x151 = wines_s.loc[151, x_vars]
np.sqrt(((x151 - x553) ** 2).sum())

In [ ]:
# Distance between 94 and 553, standardized data
x94 = wines_s.loc[94, x_vars]
np.sqrt(((x94 - x553) ** 2).sum())

Now we see that observation 94 is much closer than observation 151, which agrees with our intuition from the scatterplot.

# Exercises

**Exercise 1.** In the example above, we standardized the variables before calculating distances. How would the distances change if we instead normalized the variables? Or if we instead applied a min/max scaling to the variables?

In [ ]:
# ENTER YOUR CODE HERE

**Exercise 2.** Which wine is most similar to the wine in row 267? Use all of the variables, and see how sensitive your answer is to the choice of distance metric and standardization method?

_Hint:_ You can do this without a `for` loop. Take advantage of broadcasting!

In [ ]:
# ENTER YOUR CODE HERE